# Risk-free rates

For this exercise, I was not given the risk-free rate curves or yield curves, but they are required for some models, such as Heston. Here, I attempt to solve for the risk-free rate and yield via MSE minimization using the data I was given. In a real-life case, this would not be necessary - risk-free rate and yields can be found in other databases.

In [10]:
# Common imports
import time, os
import pandas as pd
import numpy as np
root_dir = '/mnt/c/Users/Steve/implied_vol_machine_learning'

## Define pricing function

In [16]:
import numpy as np
from scipy.stats import norm

def bs_call(S, X, T, r, y, sigma):
    b = r - y
    sig_sqrt_t = sigma * np.sqrt(T)
    d1 = (np.log(S/X) + (b+sigma*sigma/2)*T)/sig_sqrt_t
    d2 = d1 - sig_sqrt_t
    opt_val = S * np.exp(-y*T) * norm.cdf(d1) - X * np.exp(-r*T) * norm.cdf(d2)
    return opt_val

print(bs_call(100, 95, 0.5, 0.1, 0.05, 0.2))

def bscall_mse(model_params, spot_prices, strikes, T, implied_vols, opt_price):
    r, y = model_params
    val = np.array([bs_call(S, X, T, r, y, sigma) for S, X, sigma in zip(spot_prices, strikes, implied_vols)]) - opt_price
    return np.sqrt((val * val).sum())
    

9.628983522021265


## Import data

In [59]:
import os
import pandas as pd
import datetime as dt

raw_data = pd.read_csv(os.path.join(root_dir, 'options_20220824.csv'))
call_data = raw_data.loc[(raw_data["Type"] == "call") & (raw_data["Ask"] < 99000.0)].copy()
call_data.loc[:, "maturity"] = (pd.to_datetime(call_data["Expiration"]) - pd.to_datetime(call_data[" DataDate"])).dt.days / 365
call_data.loc[:, "moneyness"] = call_data["Strike"] / call_data["UnderlyingPrice"]
call_data.loc[:, "ticker"] = call_data["UnderlyingSymbol"]
call_data.loc[:, "Mid"] = (call_data["Bid"]+call_data["Ask"])/2

print(call_data.iloc[0])

UnderlyingSymbol                   A
UnderlyingPrice               133.67
Exchange                           *
OptionSymbol        A220916C00060000
Blank                            NaN
Type                            call
Expiration                09/16/2022
 DataDate           08/24/2022 16:00
Strike                          60.0
Last                             0.0
Bid                             72.5
Ask                             75.1
Volume                             0
OpenInterest                       0
IV                            0.3615
Delta                         0.9997
Gamma                            0.0
Theta                        -0.8933
Vega                             0.0
Alias               A220916C00060000
maturity                    0.060274
moneyness                   0.448867
ticker                             A
Mid                             73.8
Name: 0, dtype: object


In [62]:
focus_data = call_data.loc[(call_data["maturity"] > 0.0) & (call_data["moneyness"] <= 2.5)].copy()

## Solve for the risk-free rate and yield

In [65]:
import time
from scipy.optimize import minimize

r = 0.02; y = 0.0
start_values = [r, y]
bounds = [(-1.0, 1.0), (-1.0, 1.0)]
# For the first case, SLSQP ended up fastest and just about as accurate
# Hopefully, that is true for the others
# try_methods = ['Nelder-Mead', 'Powell', 'CG', 'BFGS', 'L-BFGS-B', 'TNC', 'COBYLA', 'SLSQP', 'trust-constr'] 

costofcarry_by_tickermaturity = {}
errors = []
start = time.time()
for key, data in focus_data.groupby(['ticker', 'Expiration']):
    res = minimize(bscall_mse, start_values, bounds=bounds, args=(data['UnderlyingPrice'], data['Strike'], data['maturity'].iloc[0], data['IV'], data['Mid']), tol=1e-3, method="SLSQP")
    if res.success:
        riskfreerate, yield = res.x
        costofcarry_by_tickermaturity[key] = (riskfreerate, yield, data.shape[0])
    else:
        errors.append(key)
end = time.time()

print(f'Time to solve: {end-start}s')
    
print('Sample values: ', list(costofcarry_by_tickermaturity.items())[0:3])
print('Problem cases: ', errors)

/tmp/ipykernel_521/2239509712.py:7: RuntimeWarning: divide by zero encountered in double_scalars
  d1 = (np.log(S/X) + (b+sigma*sigma/2)*T)/sig_sqrt_t


Time to solve: 3851.913613796234s
Sample values:  [(('A', '01/19/2024'), array([ 0.02022078, -0.00022529])), (('A', '01/20/2023'), array([ 0.00470128, -0.01294581])), (('A', '02/17/2023'), array([ 0.02040406, -0.0004123 ]))]
Problem cases:  []


In [70]:
# Sanity check: see that we reached the end
print(list(costofcarry_by_tickermaturity.items())[len(costofcarry_by_tickermaturity)-1])

(('ZYXI', '11/18/2022'), array([0.72339373, 0.54527819]))


In [85]:
# Should have started with this format, but oh well
reformatted = []
for (ticker, expiration), (r, y) in costofcarry_by_tickermaturity.items():
    reformatted.append([ticker, expiration, r, y])
    
r_and_y_df = pd.DataFrame(reformatted).rename(columns={0: 'ticker', 1: 'Expiration', 2: 'RiskFreeRate', 3: 'YieldRate'})
print(r_and_y_df.iloc[0:2])

  ticker  Expiration  RiskFreeRate  YieldRate
0      A  01/19/2024      0.020221  -0.000225
1      A  01/20/2023      0.004701  -0.012946


In [88]:
# This method of getting the risk-free rate can have surprisingly large variance
# For now, treat each ticker as having its own risk-free rate
for expiration, data in r_and_y_df.groupby('Expiration'):
    print(expiration, data['RiskFreeRate'].mean(), data['RiskFreeRate'].std())

01/18/2023 0.03306957186719332 0.007819514557066354
01/19/2024 0.04772671384147566 0.2126472706014485
01/20/2023 0.01679564627060305 0.1954735581215464
01/31/2023 0.03181704751485948 0.008077687471028576
02/15/2023 0.03183828492081532 0.0055264079377277025
02/17/2023 0.0158021274658522 0.19984662976106415
03/15/2024 -0.008682736953476548 0.06451439970107108
03/17/2023 0.027967037037580567 0.16740062092735
03/22/2023 0.02746351503402972 nan
03/31/2023 0.02873657392915677 0.09975969175153923
04/19/2023 0.02790984329227206 nan
04/21/2023 0.048845855201027515 0.18660746054387928
05/19/2023 0.022892180457593985 0.02513204472880916
06/16/2023 0.024430314225071418 0.11942102011952027
06/21/2024 0.020183917835200172 0.1512824216749102
06/30/2023 0.01137076695766223 0.06893142122872249
07/21/2023 0.018899970578790018 0.014645842866933196
08/18/2023 0.03500668103042258 0.06240762365314682
08/26/2022 0.2729718887839613 0.43404052978989544
08/29/2022 0.1378812804773952 0.4017234132917661
08/30/202

In [91]:
# Save for use in other notebooks
r_and_y_df.to_csv(os.path.join(root_dir, 'riskfreerate_and_yield_by_ticker_and_maturity.csv'))

print('Done')

Done
